In [1]:
from math import exp, sqrt
import matplotlib.pyplot as plt
import random
import sys
import pandas as pd
import numpy as np
import os,errno
from mpl_toolkits.mplot3d import Axes3D
from scipy.sparse import csgraph
from numpy import linalg as LA

In [2]:
def RBF_kernel(gamma,A,B) :
    diff = np.linalg.norm(A-B)
    d = np.exp(-(diff)/(2*gamma**2))
    return d

def laplacian(num_data,X):
    gamma = 0.1
    print ("Computing Graph Laplacian")
    W = np.zeros((num_data,num_data))
    for i in range(0,num_data):
        for j in range(i+1, num_data,1):
            W[i,j] = RBF_kernel(gamma,X[i], X[j])
            W[j,i] = W[i,j] 
    
    D = np.zeros((num_data,num_data))
    for i in range(num_data):
        D[i,i] = sum(W[i])
        
    #ratio cut
    L = D -  W
    
    return L

In [42]:
def spectral_clustering(data,K,num_data):
    
    color = ["b","g","m","r","k"]
    
    L=laplacian(num_data,data)

    #calculating eigenValues, eigenVectors
    eigenValues, eigenVectors = np.linalg.eig(L)
    
    #the first K value
    smallestEigenValue =  np.argsort(eigenValues.real)
    SpecSpace =np.zeros((num_data,0))

    for i in range(0,K,1):
            SpecCoor = (np.array(eigenVectors[:,smallestEigenValue[i]]))
            SpecCoor = np.reshape(SpecCoor,(num_data,1))
            SpecSpace = np.concatenate((SpecSpace, SpecCoor), axis=1)
    
    print(SpecSpace)
    print(SpecSpace.shape)


    #random k means
    #Randomly select K points in the new data space as the centroids
    mu =  SpecSpace [random.sample(range(0, num_data), K)]


    # kmeans
    iteration = 0
    path ="plots/SpectCluster_"+ dataset_name +"_"+str(K)+"cluster"
    try:
        os.makedirs(path)
    except OSError as e:
        if e.errno != errno.EEXIST:
            raise
    for it in range (100) :
    
        #reset for the next iteration
        clusterOriginalData = [[[None] for i in range(0)] for j in range(K)]
        clusterSpecSpaceData = [[[None] for i in range(0)] for j in range(K)]

        for i in range (num_data) :
            min_dist = 10000000
            nearestK = - 1
            for k in range (K) :
                    temp_dist = sqrt(np.linalg.norm(SpecSpace[i]-mu[k]))
                    if (temp_dist < min_dist) :
                        min_dist = temp_dist
                        nearestK = k
                
            clusterOriginalData[nearestK].append (data[i])
            clusterSpecSpaceData[nearestK].append (SpecSpace[i])

        # calculate mean
   
        for k in range (K):
            plt.plot(np.array(clusterOriginalData[k])[:,0],np.array(clusterOriginalData[k])[:,1],color[k]+"o", alpha = 0.8)

      
        plt.title(dataset_name +" iter " +str(it))
        plt.axis('equal')
        plt.savefig(path+"/SpectCluster_"+ dataset_name +"_"+ str(it)+".png", bbodata_inches="tight")
    
        for k in range (K): 
            plt.plot(np.array(clusterSpecSpaceData[k])[:,0], color[k]+"o", alpha = 0.8) 
            #plt.plot(np.array(clusterSpecSpaceData[k])[:,1],color[k]+"o", alpha = 0.8) 
        plt.title(dataset_name +" iter " +str(it))
        
        plt.savefig(path+"/SpectCluster_SpecSpace"+ dataset_name +"_"+ str(it)+".png", bbodata_inches="tight") 
        plt.clf()
        
        for k in range (K): 
            #plt.plot(np.array(clusterSpecSpaceData[k])[:,0], color[k]+"o", alpha = 0.8) 
            plt.plot(np.array(clusterSpecSpaceData[k])[:,1],color[k]+"o", alpha = 0.8) 
        plt.title(dataset_name +" iter " +str(it))
        
        plt.savefig(path+"/SpectCluster_SpecSpace2"+ dataset_name +"_"+ str(it)+".png", bbodata_inches="tight") 
        plt.clf()

        new_mu = np.zeros((K,K))
    
        for k in range (K) :
            new_mu[k] = np.mean(clusterSpecSpaceData[k],axis =0)
        if ( sqrt(np.linalg.norm(new_mu-mu)) < 0.00001):
            iteration = it
            break
        mu = new_mu.copy()

        plt.clf()
    
    print ("converged after ", iteration, " interation(s)")

In [43]:
fileName = input("Enter file name:")
dataset_name= fileName[:4]
kCluster = int(input('Enter the K number of clusters :'))
data = pd.read_csv(fileName,header=None)
data = np.array(data)
num_data = data.shape[0]
strName= dataset_name
np.random.shuffle(data)
_=spectral_clustering(data,kCluster,num_data)

Enter file name:moon.txt
Enter the K number of clusters :2
Computing Graph Laplacian
[[ 0.02581989  0.02582136]
 [ 0.02581989  0.02581803]
 [ 0.02581989 -0.0258196 ]
 ...
 [ 0.02581989 -0.02582008]
 [ 0.02581989  0.02582067]
 [ 0.02581989  0.02582135]]
(1500, 2)
converged after  2  interation(s)


In [39]:
np.zeros((num_data,0))

array([], shape=(1500, 0), dtype=float64)